In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
from scipy import sparse
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from time import perf_counter
# For multiprocessing
import multiprocessing
from psutil import cpu_count
# This one, with logical=False, is better than multiprocessing.cpu_count
# https://stackoverflow.com/questions/40217873/multiprocessing-use-only-the-physical-cores

# Change the forking method used by multiprocessing, avoids errors on Mac OS Catalina
# e.g. https://github.com/matplotlib/matplotlib/issues/15410
multiprocessing.set_start_method('forkserver')

In [ ]:
plt.rcParams["figure.figsize"] = (3, 3)

# Functions related to model for odorants and receptors
From Reddy et al. 2018, the way to define an odorant is to specify a vector of binding affinities, $\vec{\kappa}$, and a vector of activation efficacies, $\vec{\eta}$. For a background, we define $n_{mix}$ such odorants and combine them into $\vec{\eta}_{mix}$ and $\vec{\kappa}_{mix}$. 

In [ ]:
from modelfcts.shen2020_habituation_model import generate_odorant, generate_background

# Functions related to the olfactory network model

In [ ]:
from modelfcts.shen2020_habituation_model import project_neural_tag, jaccard, create_sparse_proj_mat

# Functions related to habituation and background fluctuations

In [ ]:
from modelfcts.shen2020_habituation_model import time_evolve_habituation_fixed

In [ ]:
from modelfcts.shen2020_habituation_model import combine_odorants

# Habituation with fluctuating background
Define here the function that takes an initial background and weight vector, and simulates its habituation with fluctuations of the background. 

### Background fluctuation model
Given two odorants in the background, A and B, moderately fluctuate the proportion of those two odorants in the mixture, on a time scale much faster than habituation itself. For now, we don't even fluctuate the total concentration (average of the input vector), assuming there is rescaling happening somewhere in the entry layers.

Let $f \in [0, 1]$ be the fraction of odorant A in the background. We want $f$ to be on average $0.5$ and fluctuate mildly. To simplify things, I simulate it as a Gaussian process with standard deviation $\sigma_f$ on the order of $0.1$ and clip the value of $f$ if it ever reaches a value outside of $[0, 1]$. 

In [ ]:
# Splitting those two things is computationally inefficient but more flexible for early development. 
# Eventually, write specialized functions where the update is made within the habituation function
def update_fluct_proportion(bkvec, f_bk, rates, odor_pair, noises):
    """ 
    Args:
        bkvec: ORN activation vector by the background (not needed, to respect the call signature)
        f_bk: array of length 1, containing proportion of odorants
        rates: tau_f, mean_f, vari_f, Gaussian parameters and time scale of f's fluctuation
        odor_pair: extra arguments, here a pair of 2 single odorants that compose the background. 
        noises: pre-generated normal(0, 1) samples, one per component in f_bk
    """
    # Extract parameters
    tau_f, mean_f, vari_f = rates
    
    # Update f
    f_bk[0] = f_bk[0] - (f_bk[0]-mean_f)/tau_f + np.sqrt(2*vari_f/tau_f) * noises[0]
    f_bk = np.clip(f_bk, 0, 1)
    
    # Compute new background vector
    bkvec = combine_odorants(odor_pair[0], odor_pair[1], f_bk[0])
    return bkvec, f_bk

def habituate_fluct_background(w_vec0, back_vec0, state_vec0, nstep, learnrates, bfluct_rates, rndgen, 
                               bfluct_args=(), bk_update=update_fluct_proportion):
    """ 
    Args:
        back_vec0: initial background vector of ORN activations
        state_vec0: initial background state vector (e.g. concentration, proportions)
        bk_update: general function called at each time step to update the background (Markov process). 
            Call signature: back_vec, state_vec, bfluct_rates, bfluct_args, noises
            where back_vec is the current background vector, state_vec is a vector of other background
            properties, e.g. concentration and proportions, and bfluct_params is a tuple
            of other arguments needed by the chosen bk_update
        rndgen (np.random.Generator): the random generator
        bfluct_rates: tuple of rates for the stochastic process of the background fluctuation
        bfluct_params: other parameters for the background fluctuations

    """
    # Extract parameters
    if nstep > 1e6:
        raise ValueError("Consider asking for less than 1e6 steps at a time")
    alpha, beta = learnrates
    
    # Initialize variables
    w_vec = w_vec0.copy()
    t = 0  # number of steps taken
    
    # Initial the background properties (do not modify in-place)
    back_vec = back_vec0.copy()
    state_vec = state_vec0.copy()
    state_course = np.zeros((nstep, state_vec.shape[0]))
    
    # Pre-generate normal samples for each time step (assuming a Langevin equation)
    all_noises = rndgen.normal(size=state_vec0.size*nstep).reshape(nstep, -1)
    
    # Iterate until satisfing the number of steps asked for
    while t < nstep:
        # Update the background
        back_vec, state_vec = bk_update(back_vec, state_vec, bfluct_rates, bfluct_args, all_noises[t])
        state_course[t] = state_vec
        # Update w, remove the max(s-w, 0) thing for the update rule, see if it still works (it should)
        #x_vec = np.maximum(back_vec - w_vec, 0)
        #w_vec = w_vec + alpha * x_vec - beta* w_vec
        w_vec = w_vec + alpha * back_vec - (alpha + beta)* w_vec
        t += 1

    return w_vec, back_vec, state_course

## Test 1: introducing a new pure odor after habituating to a background
This gives as much chance as possible to the habituation scheme, because the background is completely replaced with a new odor

In [ ]:
def mean_abs_deviation(x):
    return np.mean(np.abs(x - np.mean(x)))

In [ ]:
# Generate a list of odors
def similarity_habituation_proportion(all_odors=None, n_odors=100, n_orn=50, 
        n_kc=2000, n_pn_per_kc=6, steps=50, nreps=10, nbacks=10, learnrates=(0.05, 0.01), 
        bfluct_rates=(4, 0.5, 0.1), adapt_kc=True, fix_thresh=None, seed=48225):
    """Using update_fluct_proportion as the background fluctuation, habituate nreps times
    to nbacks randomly selected pairs of odorants as a background, for each odorant in the set. 
    """
    
    # Initialize projection matrix, odors, containers for scores
    rdgen_mix = np.random.default_rng(seed=seed)
    if all_odors is None:
        all_odors = np.vstack([generate_odorant(n_orn, rdgen_mix) for i in range(n_odors)])
    else:
        n_odors = all_odors.shape[1]
        n_orn = all_odors.shape[0]
        all_odors = all_odors.values.T  # Each row is an odorant now
    proj_mat = create_sparse_proj_mat(n_kc=n_kc, n_rec=n_orn, rgen=rdgen_mix, fraction_filled=n_pn_per_kc/n_orn)
    simil_before = np.zeros([n_odors, nbacks, nreps])  # J(odor, s'') before
    simil_after = np.zeros([n_odors, nbacks, nreps])   # J(odor, s'') after
    
    # Don't save the backgrounds or their time courses for now.
    # Just save the last one for each odorant actually, to check it's indeed the desired process
    last_conc_runs = np.zeros((n_odors, steps))
    
    # For each odor, habituate to a random choice of a pair of odors, nback times
    # and run nruns time each background, because the time course is stochastic.
    wzero = np.zeros(n_orn)
    f_vec0 = 0.5*np.ones(1)
    projtag_kwargs = dict(kc_sparsity=0.05, adapt_kc=adapt_kc, n_pn_per_kc=n_pn_per_kc, fix_thresh=fix_thresh)
    for i in range(n_odors):
        odi = all_odors[i]
        tag_i_alone = project_neural_tag(odi, wzero, proj_mat, **projtag_kwargs)
        other_odors = list(range(n_odors))
        other_odors.remove(i)
        for j in range(nbacks):
            # Select a background randomly
            od1, od2 = all_odors[rdgen_mix.choice(other_odors, replace=False, size=2).astype(int)]
            back_odor0 = combine_odorants(od1, od2, 0.5)
            w_vec0 = np.zeros(n_orn)
            # Compute tag of odi mixed with that background, prior to habituation
            odmix = combine_odorants(back_odor0, odi, 0.8)
            tag_mix_before = project_neural_tag(odmix, w_vec0, proj_mat, **projtag_kwargs)
            # Make k runs with that background
            for k in range(nreps):
                w_vec, back_odor, f_vec = habituate_fluct_background(w_vec0, 
                    back_odor0, f_vec0, steps, learnrates, bfluct_rates,  
                    rdgen_mix, bfluct_args=(od1, od2), bk_update=update_fluct_proportion)
                # Compute the tag of odor i after habituation to this background, 
                # mixed with back_vec
                odmix = combine_odorants(back_odor, odi, 0.8)
                tag_mix_after = project_neural_tag(odmix, w_vec, proj_mat, **projtag_kwargs)
                
                # Compute the Jaccard of i's tag alone with the mix before and after habituation
                simil_before[i, j, k] = jaccard(tag_i_alone, tag_mix_before)
                simil_after[i, j, k] = jaccard(tag_i_alone, tag_mix_after)
                
        # Before moving to next odorant, save latest f time course
        last_conc_runs[i] = f_vec.flatten()
    
    return simil_before, simil_after, last_conc_runs

In [ ]:
start_t = perf_counter()
n_odors = 100
sim_mats = similarity_habituation_proportion(all_odors=None, n_odors=20, n_orn=50, 
        n_kc=2000, n_pn_per_kc=6, steps=50, nreps=10, nbacks=10, learnrates=(0.05, 0.01), 
        bfluct_rates=(4, 0.5, 0.04), adapt_kc=True, fix_thresh=None, seed=48225)
end_t = perf_counter()
print("Time per run:", 1000*(end_t - start_t)/sim_mats[0].size, "ms")

In [ ]:
def plot_before_after(samp_before, samp_after, figax=None):
    if figax is None:
        fig, ax = plt.subplots()
    else:
        fig, ax = figax

    barcolors = sns.color_palette("mako", n_colors=4)[1:3]
    yerr = [mean_abs_deviation(samp_before), mean_abs_deviation(samp_after)]
    median_before = np.median(samp_before)
    median_after = np.median(samp_after)
    print("Median before:", median_before)
    print("Median_after:", median_after)
    ax.bar(0, median_before, yerr=yerr[0], facecolor=barcolors[0], edgecolor="k", alpha=0.5)
    ax.bar(1, median_after, yerr=yerr[1], facecolor=barcolors[1], edgecolor="k", alpha=0.5)
    ax.scatter(0.075*np.random.normal(size=samp_before.size), samp_before, s=2,
                    color=barcolors[0], alpha=0.7, lw=0.5)
    ax.scatter(1+0.075*np.random.normal(size=samp_after.size), samp_after, s=2,
                    color=barcolors[1], alpha=0.8, lw=0.5)
    ax.set_title(r"$s_A(0)$ vs $s_{mix}(t)$")
    ax.set_ylabel("Jaccard similarity")

    ylims = ax.get_ylim()
    ax.set_ylim(ylims[0], ylims[1]+0.1)
    ax.annotate("Before", xy=(0, ylims[1]+0.05), ha="center", va="top", fontsize=8)
    ax.annotate("After", xy=(1, ylims[1]+0.05), ha="center", va="top", fontsize=8)
    ax.set_xticks([0, 1])
    ax.set_xticklabels([r"$t=0$", r"$t=50$"])
    ax.set_xlabel("Habituation")

    return [fig, ax]

In [ ]:
samples_before = sim_mats[0].flatten()
samples_after = sim_mats[1].flatten()

# Prepare two barplots: one for A vs mix before and after, one for B vs mix before and after
fig, ax = plot_before_after(samples_before, samples_after, figax=None)
    
fig.tight_layout()
#fig.savefig("figures/shen2020_related/habituation_fluctuating_proportion.pdf", transparent=True)

plt.show()
plt.close()

In [ ]:
fig, axes = plt.subplots(1, 2)
fig.set_size_inches(6, 3)
axes = axes.flatten()

times = np.arange(sim_mats[2].shape[1])
colors = sns.color_palette("mako", n_colors=sim_mats[2].shape[0])
for i in range(sim_mats[2].shape[0]):
    axes[0].plot(times, sim_mats[2][i], lw=1., color=colors[i])
axes[0].set(xlabel="Time steps", ylabel=r"$f$")

axes[1].hist(sim_mats[2].flatten(), linewidth=1., edgecolor="k", facecolor=colors[len(colors)//2])
axes[1].set(xlabel=r"$f$", ylabel="Frequency")
fig.tight_layout()
# fig.savefig("figures/shen2020_related/composition_fluctuations.pdf", transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
np.var(sim_mats[2])

## Habituation to changing total concentration and composition
Change proportion and total concentration independently. 


In [ ]:
def update_fluct_proportion_conc(bkvec, fc_bk, rates, odor_pair, noises):
    """ 
    Args:
        bkvec: ORN activation vector by the background (not needed, to respect the call signature)
        fc_bk: array of length 2, containing proportion of odorants and total concentration (average 10)
        rates: tau_f, mean_f, vari_f, tau_c, mean_c, vari_c: 
            Gaussian parameters and time scale of f and c fluctuation
        odor_pair: extra arguments, here a pair of 2 single odorants that compose the background. 
        noises: pre-generated normal(0, 1) samples, one per component in f_bk
    """
    # Extract parameters
    tau_f, mean_f, vari_f, tau_c, mean_c, vari_c = rates
    
    # Update f
    fc_bk[0] = fc_bk[0] - (fc_bk[0]-mean_f)/tau_f + np.sqrt(2*vari_f/tau_f) * noises[0]
    fc_bk[1] = fc_bk[1] - (fc_bk[1]-mean_c)/tau_c + np.sqrt(2*vari_c/tau_c) * noises[1]
    fc_bk[0] = np.clip(fc_bk[0], 0, 1)
    fc_bk[1] = np.clip(fc_bk[1], a_min=0, a_max=np.inf)  # keep the concentration non-negative
    
    # Compute new background vector
    bkvec = combine_odorants(odor_pair[0], odor_pair[1], fc_bk[0]) * fc_bk[1] / mean_c
    return bkvec, fc_bk

In [ ]:
# Generate a list of odors
def similarity_habituation_proportion_conc(all_odors=None, n_odors=100, n_orn=50, 
        n_kc=2000, n_pn_per_kc=6, steps=50, nreps=10, nbacks=10, learnrates=(0.05, 0.01), 
        bfluct_rates=(4, 0.5, 0.1, 4, 10, 4), adapt_kc=False, fix_thresh=20, seed=4822537):
    """Using update_fluct_proportion as the background fluctuation, habituate nreps times
    to nbacks randomly selected pairs of odorants as a background, for each odorant in the set. 
    """
    # Initialize projection matrix, odors, containers for scores
    fix_thresh = bfluct_rates[-2]  # Average concentration should be the fixed threshold
    rdgen_mix = np.random.default_rng(seed=seed)
    if all_odors is None:
        # Normalize each odor to be equal to the fixed threshold
        all_odors = np.vstack([generate_odorant(n_orn, rdgen_mix) for i in range(n_odors)])
    else:
        n_odors = all_odors.shape[1]
        n_orn = all_odors.shape[0]
        all_odors = all_odors.values.T  # Each row is an odorant now
    proj_mat = create_sparse_proj_mat(n_kc=n_kc, n_rec=n_orn, rgen=rdgen_mix, fraction_filled=n_pn_per_kc/n_orn)
    simil_before = np.zeros([n_odors, nbacks, nreps])  # J(odor, s'') before
    simil_after = np.zeros([n_odors, nbacks, nreps])   # J(odor, s'') after
    
    # Don't save the backgrounds or their time courses for now.
    # Just save the last one for each odorant actually, to check it's indeed the desired process
    last_conc_runs = np.zeros((n_odors, steps, 2))
    
    # For each odor, habituate to a random choice of a pair of odors, nback times
    # and run nruns time each background, because the time course is stochastic.
    wzero = np.zeros(n_orn)
    projtag_kwargs = dict(kc_sparsity=0.05, adapt_kc=adapt_kc, n_pn_per_kc=n_pn_per_kc, fix_thresh=fix_thresh)
    fc_vec0 = np.asarray([bfluct_rates[1], bfluct_rates[-2]])
    for i in range(n_odors):
        odi = all_odors[i]
        tag_i_alone = project_neural_tag(odi, wzero, proj_mat, **projtag_kwargs)
        other_odors = list(range(n_odors))
        other_odors.remove(i)
        for j in range(nbacks):
            # Select a background randomly
            od1, od2 = all_odors[rdgen_mix.choice(other_odors, replace=False, size=2).astype(int)]
            back_odor0 = combine_odorants(od1, od2, 0.5)
            w_vec0 = np.zeros(n_orn)
            # Compute tag of odi mixed with that background, prior to habituation
            odmix = combine_odorants(back_odor0, odi, 0.8)
            tag_mix_before = project_neural_tag(odmix, w_vec0, proj_mat, **projtag_kwargs)
            # Make k runs with that background
            for k in range(nreps):
                w_vec, back_odor, fc_vec = habituate_fluct_background(w_vec0, 
                    back_odor0, fc_vec0, steps, learnrates, bfluct_rates,  
                    rdgen_mix, bfluct_args=(od1, od2), bk_update=update_fluct_proportion_conc)
                # Compute the tag of odor i after habituation to this background, 
                # mixed with back_vec
                odmix = combine_odorants(back_odor, odi, 0.8)
                tag_mix_after = project_neural_tag(odmix, w_vec, proj_mat, **projtag_kwargs)
                
                # Compute the Jaccard of i's tag alone with the mix before and after habituation
                simil_before[i, j, k] = jaccard(tag_i_alone, tag_mix_before)
                simil_after[i, j, k] = jaccard(tag_i_alone, tag_mix_after)
                
        # Before moving to next odorant, save latest f time course
        last_conc_runs[i] = fc_vec
    
    return simil_before, simil_after, last_conc_runs

In [ ]:
start_t = perf_counter()
n_odors = 100
sim_mats_c = similarity_habituation_proportion_conc(all_odors=None, n_odors=20, n_orn=50, 
        n_kc=2000, n_pn_per_kc=6, steps=50, nreps=10, nbacks=10, learnrates=(0.05, 0.01), 
        bfluct_rates=(4, 0.5, 0.04, 4, 10, 4), adapt_kc=False, fix_thresh=20, seed=4822537)
end_t = perf_counter()
print("Time per run:", 1000*(end_t - start_t)/sim_mats_c[0].size, "ms")

In [ ]:
samples_before = sim_mats_c[0].flatten()
samples_after = sim_mats_c[1].flatten()

# Prepare two barplots: one for A vs mix before and after, one for B vs mix before and after
fig, ax = plot_before_after(samples_before, samples_after, figax=None)
    
fig.tight_layout()
fig.savefig("figures/habituation_fluctuating_composition_concentration.pdf", transparent=True, bbox_inches="tight")

plt.show()
plt.close()

In [ ]:
fig, axes = plt.subplots(1, 2)
fig.set_size_inches(6, 3)
axes = axes.flatten()

times = np.arange(sim_mats_c[2].shape[1])
colors = sns.color_palette("mako", n_colors=sim_mats_c[2].shape[0])
for i in range(sim_mats_c[2].shape[0]):
    axes[0].plot(times, sim_mats_c[2][i, :, 1], lw=1., color=colors[i])
axes[0].set(xlabel="Time steps", ylabel=r"$C$")

axes[1].hist(sim_mats_c[2][:, :, 1].flatten(), linewidth=1., edgecolor="k", facecolor=colors[len(colors)//2])
axes[1].set(xlabel=r"$C$", ylabel="Frequency")
fig.tight_layout()
fig.savefig("figures/concentration_fluctuations_check.pdf", transparent=True)
plt.show()
plt.close()

## Next idea: change the number of odorants in the background
If we have $N$ odorants int he background, we simulate $N$ independent gaussian stochastic processes for the fractions $f_i$ of odorants, and we normalize their sum to 1 when we compute the mixture background vector. 
Or, if we wanted the total concentration to fluctuate too, we could just not normalize.

In [ ]:
def combine_many_odorants(odlist, props):
    total_f = np.sum(props)
    if total_f > 0:
        props = props / total_f
    else:
        props = np.zeros(odlist.shape[0])
    return np.sum(props[:, np.newaxis] * odlist, axis=0)

In [ ]:
def update_fluct_manybk_conc(bkvec, fc_bk, rates, odor_array, noises):
    """ 
    Args:
        bkvec: ORN activation vector by the background (not needed, to respect the call signature)
        fc_bk: array of length n_in_bk-2, containing proportion of odorants and total concentration (average 10)
        rates: tau_f, mean_f, vari_f, tau_c, mean_c, vari_c: 
            Gaussian parameters and time scale of f and c fluctuation
        odor_array: extra arguments, here an array where each row is one of the odorants in the background
        noises: pre-generated normal(0, 1) samples, one per component in f_bk
    """
    # Extract parameters. Assume same tau, mean and variance for all f's
    tau_f, mean_f, vari_f, tau_c, mean_c, vari_c = rates
    # Assume that odor_array.shape[0] = fc_bk.shape[0]-1, otherwise problems!
    
    # Update proportions f and concentration c
    fc_bk[:-1] = fc_bk[:-1] - (fc_bk[:-1]-mean_f)/tau_f + np.sqrt(2*vari_f/tau_f) * noises[:-1]
    fc_bk[-1] = fc_bk[-1] - (fc_bk[-1]-mean_c)/tau_c + np.sqrt(2*vari_c/tau_c) * noises[-1]
    
    fc_bk[:-1] = np.clip(fc_bk[:-1], 0, 1)
    fc_bk[-1] = np.clip(fc_bk[-1], a_min=0, a_max=np.inf)  # keep the concentration non-negative
    
    # Compute new background vector
    # The total concentration changes independently, the same way for all odorants, 
    # because they are all carried by the same wind. 
    bkvec = combine_many_odorants(odor_array, fc_bk[:-1]) * fc_bk[-1] / mean_c
    return bkvec, fc_bk

In [ ]:
# Generate a list of odors
def similarity_habituation_mixback_conc(all_odors=None, n_odors=20, n_in_bk=4, n_orn=50, 
        n_kc=2000, n_pn_per_kc=6, steps=50, nreps=10, nbacks=10, learnrates=(0.05, 0.01), 
        bfluct_rates=(4, 0.5, 0.1, 4, 10, 4), adapt_kc=False, fix_thresh=20, seed=4822537):
    """Using update_fluct_proportion as the background fluctuation, habituate nreps times
    to nbacks randomly selected pairs of odorants as a background, for each odorant in the set. 
    """
    # Initialize projection matrix, odors, containers for scores
    fix_thresh = bfluct_rates[-2]  # Average concentration should be the fixed threshold
    rdgen_mix = np.random.default_rng(seed=seed)
    if n_in_bk > n_odors-1:
        n_in_bk = n_odors-1
    if all_odors is None:
        # Normalize each odor to be equal to the fixed threshold
        all_odors = np.vstack([generate_odorant(n_orn, rdgen_mix) for i in range(n_odors)])
    else:
        n_odors = all_odors.shape[1]
        n_orn = all_odors.shape[0]
        all_odors = all_odors.values.T  # Each row is an odorant now
    proj_mat = create_sparse_proj_mat(n_kc=n_kc, n_rec=n_orn, rgen=rdgen_mix, fraction_filled=n_pn_per_kc/n_orn)
    simil_before = np.zeros([n_odors, nbacks, nreps])  # J(odor, s'') before
    simil_after = np.zeros([n_odors, nbacks, nreps])   # J(odor, s'') after
    
    # Don't save the backgrounds or their time courses for now.
    # Just save the last one for each odorant actually, to check it's indeed the desired process
    last_conc_runs = np.zeros((n_odors, steps, n_in_bk+1))
    
    # For each odor, habituate to a random choice of background odors, nback times
    # and run nruns time each background, because the time course is stochastic.
    wzero = np.zeros(n_orn)
    fc_vec0 = np.asarray([bfluct_rates[1]]*n_in_bk + [bfluct_rates[-2]])
    projtag_kwargs = dict(kc_sparsity=0.05, adapt_kc=adapt_kc, n_pn_per_kc=n_pn_per_kc, fix_thresh=fix_thresh)
    for i in range(n_odors):
        odi = all_odors[i]
        tag_i_alone = project_neural_tag(odi, wzero, proj_mat, **projtag_kwargs)
        other_odors = list(range(n_odors))
        other_odors.remove(i)
        for j in range(nbacks):
            # Select a background randomly
            odarray = all_odors[rdgen_mix.choice(other_odors, replace=False, size=n_in_bk).astype(int)]
            w_vec0 = np.zeros(n_orn)
            back_odor0 = combine_many_odorants(odarray, fc_vec0[:-1])
            # Compute tag of odi mixed with that background, prior to habituation
            odmix = combine_odorants(back_odor0, odi, 0.8)
            tag_mix_before = project_neural_tag(odmix, w_vec0, proj_mat, **projtag_kwargs)
            # Make k runs with that background
            for k in range(nreps):
                w_vec, back_odor, fc_course = habituate_fluct_background(w_vec0, 
                    back_odor0, fc_vec0, steps, learnrates, bfluct_rates,  
                    rdgen_mix, bfluct_args=odarray, bk_update=update_fluct_manybk_conc)
                # Compute the tag of odor i after habituation to this background, 
                # mixed with back_vec
                odmix = combine_odorants(back_odor, odi, 0.8)
                tag_mix_after = project_neural_tag(odmix, w_vec, proj_mat, **projtag_kwargs)
                # Compute the Jaccard of i's tag alone with the mix before and after habituation
                simil_before[i, j, k] = jaccard(tag_i_alone, tag_mix_before)
                simil_after[i, j, k] = jaccard(tag_i_alone, tag_mix_after)
                
        # Before moving to next odorant, save latest f time course
        last_conc_runs[i] = fc_course
    
    return simil_before, simil_after, last_conc_runs

In [ ]:
sim_mats_c3 = similarity_habituation_mixback_conc(all_odors=None, n_odors=n_odors, n_in_bk=2, n_orn=50, 
            n_kc=2000, n_pn_per_kc=6, steps=50, nreps=10, nbacks=10, learnrates=(0.05, 0.01), 
            bfluct_rates=(4, 0.5, 0.04, 4, 10, 9), adapt_kc=False, fix_thresh=20, seed=4237)

In [ ]:
# Try this for different numbers of odors in the background
all_sim_mats = [sim_mats]
n_odors = 20
n_in_back_list = [2, 4, 6, 8, 12, 16]
for n in n_in_back_list[1:]:
    start_t = perf_counter()
    sim_mats_c2 = similarity_habituation_mixback_conc(all_odors=None, n_odors=n_odors, n_in_bk=n, n_orn=50, 
            n_kc=2000, n_pn_per_kc=6, steps=50, nreps=10, nbacks=10, learnrates=(0.05, 0.01), 
            bfluct_rates=(4, 0.5, 0.04, 4, 10, 4), adapt_kc=False, fix_thresh=20, seed=4237+n)
    end_t = perf_counter()
    print("Time per run:", 1000*(end_t - start_t)/sim_mats_c2[0].size, "ms")
    all_sim_mats.append(sim_mats_c2)

In [ ]:
# Prepare a panel of plots for the different numbers of odorants in the background
fig, axes = plt.subplots(2, 3)
axes = axes.flatten()
fig.set_size_inches(3*3, 2*3)
for i in range(len(n_in_back_list)):
    samples_before = all_sim_mats[i][0].flatten()
    samples_after = all_sim_mats[i][1].flatten()

    # Prepare two barplots: one for A vs mix before and after, one for B vs mix before and after
    fig, axes[i] = plot_before_after(samples_before, samples_after, figax=[fig, axes[i]])
    axes[i].set_title(r"$n_{back} = " + r"{}$".format(n_in_back_list[i]))
    
fig.tight_layout()
fig.savefig("figures/habituation_background_complexity.pdf", transparent=True, bbox_inches="tight")

plt.show()
plt.close()